In [1]:
import pandas as pd
import numpy as np

In [816]:
YEAR = 2038

In [817]:
vehicles_df = pd.read_csv('data/vehicles.csv')
demand_df = pd.read_csv('data/demand.csv')

In [818]:
vehicles_df

,ID,Vehicle,Size,Year,Cost ($),Yearly range (km),Distance
0,BEV_S1_2023,BEV,S1,2023,187000,102000,D1
1,BEV_S1_2024,BEV,S1,2024,177650,102000,D1
2,BEV_S1_2025,BEV,S1,2025,168767,102000,D1
3,BEV_S1_2026,BEV,S1,2026,160329,102000,D2
4,BEV_S1_2027,BEV,S1,2027,152312,102000,D2
...,...,...,...,...,...,...,...
187,LNG_S3_2034,LNG,S3,2034,209208,73000,D4
188,LNG_S3_2035,LNG,S3,2035,215484,73000,D4
189,LNG_S3_2036,LNG,S3,2036,221948,73000,D4
190,LNG_S3_2037,LNG,S3,2037,228607,73000,D4


In [819]:
demand_df

,Year,Size,Distance,Demand (km)
0,2023,S1,D1,869181
1,2023,S1,D2,2597094
2,2023,S1,D3,3292011
3,2023,S1,D4,414315
4,2023,S2,D1,995694
...,...,...,...,...
251,2038,S4,D4,2446
252,2038,S3,D1,3229278
253,2038,S3,D2,3297618
254,2038,S3,D3,1448550


In [820]:
def allocate_vehicles(demand_df, vehicles_df, year):
    # Mapping for categorical distance values
    distance_mapping = {"D1": 1, "D2": 2, "D3": 3, "D4": 4}
    
    # Convert categorical distance to numeric ranks
    vehicles_df['Distance_categorical'] = vehicles_df['Distance'].map(distance_mapping)
    demand_df['Distance_categorical'] = demand_df['Distance'].map(distance_mapping)

    demand_df = demand_df[demand_df['Year'] == year].copy() # Filter relevant year
    available_vehicles = vehicles_df[vehicles_df['Year'] == year]  # Vehicles available in that year

    def find_allocations(row):
        size_needed = row['Size']
        distance_needed = row['Distance_categorical']
        
        allocated_vehicles = available_vehicles[
            (available_vehicles['Size'] == size_needed) & 
            (available_vehicles['Distance_categorical'] >= distance_needed)
        ]['ID'].tolist() 
        
        return allocated_vehicles

    demand_df['Allocation'] = demand_df.apply(find_allocations, axis=1)  # Apply function row-wise
    
    return demand_df


In [821]:
allocation_df = allocate_vehicles(demand_df, vehicles_df, YEAR)
allocation_df.to_csv('data/allocation_result.csv', index=False)

In [822]:
allocation_df

,Year,Size,Distance,Demand (km),Distance_categorical,Allocation
240,2038,S1,D1,1254358,1,"[BEV_S1_2038, Diesel_S1_2038, LNG_S1_2038]"
241,2038,S1,D2,3994169,2,"[BEV_S1_2038, Diesel_S1_2038, LNG_S1_2038]"
242,2038,S1,D3,4847871,3,"[BEV_S1_2038, Diesel_S1_2038, LNG_S1_2038]"
243,2038,S1,D4,642216,4,"[BEV_S1_2038, Diesel_S1_2038, LNG_S1_2038]"
244,2038,S2,D1,1410693,1,"[BEV_S2_2038, Diesel_S2_2038, LNG_S2_2038]"
245,2038,S2,D2,2151804,2,"[BEV_S2_2038, Diesel_S2_2038, LNG_S2_2038]"
246,2038,S2,D3,1187938,3,"[BEV_S2_2038, Diesel_S2_2038, LNG_S2_2038]"
247,2038,S2,D4,190889,4,"[BEV_S2_2038, Diesel_S2_2038, LNG_S2_2038]"
248,2038,S4,D1,23437,1,"[BEV_S4_2038, Diesel_S4_2038, LNG_S4_2038]"
249,2038,S4,D2,1148555,2,"[BEV_S4_2038, Diesel_S4_2038, LNG_S4_2038]"


In [823]:
df_exploded = allocation_df.explode("Allocation")

In [824]:
# Display the transformed DataFrame
print(df_exploded)

     Year Size Distance  Demand (km)  Distance_categorical      Allocation
240  2038   S1       D1      1254358                     1     BEV_S1_2038
240  2038   S1       D1      1254358                     1  Diesel_S1_2038
240  2038   S1       D1      1254358                     1     LNG_S1_2038
241  2038   S1       D2      3994169                     2     BEV_S1_2038
241  2038   S1       D2      3994169                     2  Diesel_S1_2038
241  2038   S1       D2      3994169                     2     LNG_S1_2038
242  2038   S1       D3      4847871                     3     BEV_S1_2038
242  2038   S1       D3      4847871                     3  Diesel_S1_2038
242  2038   S1       D3      4847871                     3     LNG_S1_2038
243  2038   S1       D4       642216                     4     BEV_S1_2038
243  2038   S1       D4       642216                     4  Diesel_S1_2038
243  2038   S1       D4       642216                     4     LNG_S1_2038
244  2038   S2       D1  

In [825]:
df = pd.merge(df_exploded[['Allocation', 'Year', 'Size', 'Distance', 'Demand (km)']], vehicles_df[['ID','Vehicle', 'Year', 'Cost ($)', 'Yearly range (km)', 'Distance']], how='left', left_on='Allocation', right_on='ID')

In [826]:
df = df.rename(columns={'Year_x': 'Operating Year', 'Year_y': 'Available Year', 'Distance_x': 'Distance_demand', 'Distance_y': 'Distance_vehicle'})

In [827]:
df.columns

Index(['Allocation', 'Operating Year', 'Size', 'Distance_demand',
       'Demand (km)', 'ID', 'Vehicle', 'Available Year', 'Cost ($)',
       'Yearly range (km)', 'Distance_vehicle'],
      dtype='object')

In [828]:
vehicle_fuels_df = pd.read_csv('data/vehicles_fuels.csv')

In [829]:
df = pd.merge(df, vehicle_fuels_df, how='left', left_on='ID', right_on='ID')

In [830]:
df

,Allocation,Operating Year,Size,Distance_demand,Demand (km),ID,Vehicle,Available Year,Cost ($),Yearly range (km),Distance_vehicle,Fuel,Consumption (unit_fuel/km)
0,BEV_S1_2038,2038,S1,D1,1254358,BEV_S1_2038,BEV,2038,130589,102000,D4,Electricity,0.868161
1,Diesel_S1_2038,2038,S1,D1,1254358,Diesel_S1_2038,Diesel,2038,132427,102000,D4,B20,0.223016
2,Diesel_S1_2038,2038,S1,D1,1254358,Diesel_S1_2038,Diesel,2038,132427,102000,D4,HVO,0.223016
3,LNG_S1_2038,2038,S1,D1,1254358,LNG_S1_2038,LNG,2038,155796,102000,D4,LNG,0.154735
4,LNG_S1_2038,2038,S1,D1,1254358,LNG_S1_2038,LNG,2038,155796,102000,D4,BioLNG,0.154381
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,BEV_S3_2038,2038,S3,D4,306910,BEV_S3_2038,BEV,2038,215849,73000,D4,Electricity,0.859516
76,Diesel_S3_2038,2038,S3,D4,306910,Diesel_S3_2038,Diesel,2038,184123,73000,D4,B20,0.224082
77,Diesel_S3_2038,2038,S3,D4,306910,Diesel_S3_2038,Diesel,2038,184123,73000,D4,HVO,0.224082
78,LNG_S3_2038,2038,S3,D4,306910,LNG_S3_2038,LNG,2038,235465,73000,D4,LNG,0.153945


In [831]:
df.drop('Consumption (unit_fuel/km)', axis=1, inplace=True)

In [832]:
df

,Allocation,Operating Year,Size,Distance_demand,Demand (km),ID,Vehicle,Available Year,Cost ($),Yearly range (km),Distance_vehicle,Fuel
0,BEV_S1_2038,2038,S1,D1,1254358,BEV_S1_2038,BEV,2038,130589,102000,D4,Electricity
1,Diesel_S1_2038,2038,S1,D1,1254358,Diesel_S1_2038,Diesel,2038,132427,102000,D4,B20
2,Diesel_S1_2038,2038,S1,D1,1254358,Diesel_S1_2038,Diesel,2038,132427,102000,D4,HVO
3,LNG_S1_2038,2038,S1,D1,1254358,LNG_S1_2038,LNG,2038,155796,102000,D4,LNG
4,LNG_S1_2038,2038,S1,D1,1254358,LNG_S1_2038,LNG,2038,155796,102000,D4,BioLNG
...,...,...,...,...,...,...,...,...,...,...,...,...
75,BEV_S3_2038,2038,S3,D4,306910,BEV_S3_2038,BEV,2038,215849,73000,D4,Electricity
76,Diesel_S3_2038,2038,S3,D4,306910,Diesel_S3_2038,Diesel,2038,184123,73000,D4,B20
77,Diesel_S3_2038,2038,S3,D4,306910,Diesel_S3_2038,Diesel,2038,184123,73000,D4,HVO
78,LNG_S3_2038,2038,S3,D4,306910,LNG_S3_2038,LNG,2038,235465,73000,D4,LNG


In [833]:
from utilities.my_sql_operations import MySQLOperations
import pandas as pd

In [834]:
def per_km_carbon_emmissions_per_vehicle(fleet_details: pd.DataFrame, op_year: int):
        """
        Calculates total carbon emissions of the fleet
        """     
        my_sql_operations = MySQLOperations() 
        
        query = f"""SELECT fuel, emissions_co2_per_unit_fuel FROM fuels WHERE year = {op_year}"""
        fuel_data, columns = my_sql_operations.fetch_data(query) 
        fuel_df = pd.DataFrame(fuel_data, columns=columns)
        
        query = f"""SELECT * FROM vehicles_fuels"""
        vehicles_fuels_data, columns = my_sql_operations.fetch_data(query)
        vehicles_fuels_df = pd.DataFrame(vehicles_fuels_data, columns=columns)

        merged_df = pd.merge(
            pd.merge(fleet_details, fuel_df, left_on='Fuel', right_on='fuel', how='left'),
            vehicles_fuels_df, left_on=['ID', 'Fuel'], right_on=['id', 'fuel'], how='left'
        )
        
        merged_df['carbon_emissions_per_km'] = (
            # merged_df['Yearly range (km)'] * 
            1 * # No. of vehicles
            merged_df['emissions_co2_per_unit_fuel'] *
            merged_df['consumption_unitfuel_per_km']
        )
        return merged_df['carbon_emissions_per_km']

In [835]:
df['carbon_emissions_per_km'] = per_km_carbon_emmissions_per_vehicle(df, YEAR)

In [836]:
df

,Allocation,Operating Year,Size,Distance_demand,Demand (km),ID,Vehicle,Available Year,Cost ($),Yearly range (km),Distance_vehicle,Fuel,carbon_emissions_per_km
0,BEV_S1_2038,2038,S1,D1,1254358,BEV_S1_2038,BEV,2038,130589,102000,D4,Electricity,0.000000
1,Diesel_S1_2038,2038,S1,D1,1254358,Diesel_S1_2038,Diesel,2038,132427,102000,D4,B20,0.679882
2,Diesel_S1_2038,2038,S1,D1,1254358,Diesel_S1_2038,Diesel,2038,132427,102000,D4,HVO,0.108748
3,LNG_S1_2038,2038,S1,D1,1254358,LNG_S1_2038,LNG,2038,155796,102000,D4,LNG,0.384701
4,LNG_S1_2038,2038,S1,D1,1254358,LNG_S1_2038,LNG,2038,155796,102000,D4,BioLNG,0.058424
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,BEV_S3_2038,2038,S3,D4,306910,BEV_S3_2038,BEV,2038,215849,73000,D4,Electricity,0.000000
76,Diesel_S3_2038,2038,S3,D4,306910,Diesel_S3_2038,Diesel,2038,184123,73000,D4,B20,0.683132
77,Diesel_S3_2038,2038,S3,D4,306910,Diesel_S3_2038,Diesel,2038,184123,73000,D4,HVO,0.109268
78,LNG_S3_2038,2038,S3,D4,306910,LNG_S3_2038,LNG,2038,235465,73000,D4,LNG,0.382737


In [837]:
def yearly_insurance_cost_per_vehicle(fleet_details: pd.DataFrame):
        """
        Returns Insurance cost for the operating year
        """
        total_fleet_insurance_cost = 0
        my_sql_operations = MySQLOperations() 
        
        query = f"""SELECT id, year, cost FROM vehicles"""
        purchase_year_data, columns = my_sql_operations.fetch_data(query) 
        purchase_year_df = pd.DataFrame(purchase_year_data, columns=columns)
        
        query = f"""SELECT end_of_year, insurance_cost_percent FROM cost_profiles"""
        insurance_cost_data, columns = my_sql_operations.fetch_data(query)
        insurance_cost_df = pd.DataFrame(insurance_cost_data, columns=columns)
        
        eoy_df = pd.merge(fleet_details, purchase_year_df, left_on=['ID'], right_on=['id'], how='left')
                
        eoy_df['End_of_year'] = (
            eoy_df['Operating Year'] - eoy_df['year'] + 1
        )
        
        merged_df = pd.merge(eoy_df, insurance_cost_df, left_on='End_of_year', right_on='end_of_year', how='left')
        
        merged_df['insurance_cost'] = (
            ((merged_df['insurance_cost_percent']/100) * merged_df['cost']) * 1 #No of vehicles 
        )
        return merged_df['insurance_cost']
            
def yearly_maintenance_cost_per_vehicle(fleet_details: pd.DataFrame):
        """
        Returns Maintenance cost for the operating year
        """
        total_fleet_maintainance_cost = 0
        my_sql_operations = MySQLOperations() 
        
        query = f"""SELECT id, year, cost FROM vehicles"""
        purchase_year_data, columns = my_sql_operations.fetch_data(query) 
        purchase_year_df = pd.DataFrame(purchase_year_data, columns=columns)
        
        query = f"""SELECT end_of_year, maintenance_cost_percent FROM cost_profiles"""
        maintenance_cost_data, columns = my_sql_operations.fetch_data(query)
        maintenance_cost_df = pd.DataFrame(maintenance_cost_data, columns=columns)
        
        eoy_df = pd.merge(fleet_details, purchase_year_df, left_on=['ID'], right_on=['id'], how='left')
                
        eoy_df['End_of_year'] = (
            eoy_df['Operating Year'] - eoy_df['year'] + 1
        )
        
        merged_df = pd.merge(eoy_df, maintenance_cost_df, left_on='End_of_year', right_on='end_of_year', how='left')
        
        merged_df['maintenance_cost'] = (
            ((merged_df['maintenance_cost_percent']/100) * merged_df['cost']) * 1 #No of vehicles 
        )
        return merged_df['maintenance_cost']
          
def per_km_fuel_cost_per_vehicle(fleet_details: pd.DataFrame, op_year: int):
        """
        Returns yearly fuel cost
        """
        yearly_fuel_cost = 0
        my_sql_operations = MySQLOperations() 
        
        query = f"""SELECT fuel, cost_per_unit_fuel FROM fuels WHERE year = {op_year}"""
        fuel_cost_data, columns = my_sql_operations.fetch_data(query) 
        fuel_cost_df = pd.DataFrame(fuel_cost_data, columns=columns)
        
        query = f"""SELECT * FROM vehicles_fuels"""
        fuel_consumption_data, columns = my_sql_operations.fetch_data(query)
        fuel_consumption_df = pd.DataFrame(fuel_consumption_data, columns=columns)
        
        merged_df = pd.merge(
            pd.merge(fleet_details, fuel_cost_df, left_on='Fuel', right_on='fuel', how='left'),
            fuel_consumption_df, left_on=['ID', 'Fuel'], right_on=['id', 'fuel'], how='left'
        )
        
        merged_df['fuel_costs_per_km'] = (
            # merged_df['Yearly range (km)'] * 
            1 * # No of vehicles 
            merged_df['consumption_unitfuel_per_km'] *
            merged_df['cost_per_unit_fuel']
        )
        return merged_df['fuel_costs_per_km']

In [838]:
df['insurance_cost'] = yearly_insurance_cost_per_vehicle(df)
df['maintenance_cost'] = yearly_maintenance_cost_per_vehicle(df)
df['fuel_costs_per_km'] = per_km_fuel_cost_per_vehicle(df, YEAR)


In [839]:
df

,Allocation,Operating Year,Size,Distance_demand,Demand (km),ID,Vehicle,Available Year,Cost ($),Yearly range (km),Distance_vehicle,Fuel,carbon_emissions_per_km,insurance_cost,maintenance_cost,fuel_costs_per_km
0,BEV_S1_2038,2038,S1,D1,1254358,BEV_S1_2038,BEV,2038,130589,102000,D4,Electricity,0.000000,6529.45,1305.89,0.090210
1,Diesel_S1_2038,2038,S1,D1,1254358,Diesel_S1_2038,Diesel,2038,132427,102000,D4,B20,0.679882,6621.35,1324.27,0.373291
2,Diesel_S1_2038,2038,S1,D1,1254358,Diesel_S1_2038,Diesel,2038,132427,102000,D4,HVO,0.108748,6621.35,1324.27,0.343929
3,LNG_S1_2038,2038,S1,D1,1254358,LNG_S1_2038,LNG,2038,155796,102000,D4,LNG,0.384701,7789.80,1557.96,0.143362
4,LNG_S1_2038,2038,S1,D1,1254358,LNG_S1_2038,LNG,2038,155796,102000,D4,BioLNG,0.058424,7789.80,1557.96,0.153808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,BEV_S3_2038,2038,S3,D4,306910,BEV_S3_2038,BEV,2038,215849,73000,D4,Electricity,0.000000,10792.45,2158.49,0.089311
76,Diesel_S3_2038,2038,S3,D4,306910,Diesel_S3_2038,Diesel,2038,184123,73000,D4,B20,0.683132,9206.15,1841.23,0.375075
77,Diesel_S3_2038,2038,S3,D4,306910,Diesel_S3_2038,Diesel,2038,184123,73000,D4,HVO,0.109268,9206.15,1841.23,0.345573
78,LNG_S3_2038,2038,S3,D4,306910,LNG_S3_2038,LNG,2038,235465,73000,D4,LNG,0.382737,11773.25,2354.65,0.142630


In [840]:
df.to_csv(f'data/topsis_input_{YEAR}.csv', index=False)

In [814]:
transaction_types = ["Buy", "Sell", "Use"]

In [815]:
df["Type"] = [transaction_types for i in df.index]

In [500]:
df_exploded = df.explode("Type")

In [501]:
df_exploded

,Allocation,Operating Year,Size,Distance_demand,Demand (km),ID,Vehicle,Available Year,Cost ($),Yearly range (km),Distance_vehicle,Fuel,carbon_emissions_per_km,insurance_cost,maintenance_cost,fuel_costs_per_km,Type
0,BEV_S1_2023,2033,S1,D1,1070915,BEV_S1_2023,BEV,2023,187000,102000,D1,Electricity,0.000000,NaN,NaN,0.113829,Buy
0,BEV_S1_2023,2033,S1,D1,1070915,BEV_S1_2023,BEV,2023,187000,102000,D1,Electricity,0.000000,NaN,NaN,0.113829,Sell
0,BEV_S1_2023,2033,S1,D1,1070915,BEV_S1_2023,BEV,2023,187000,102000,D1,Electricity,0.000000,NaN,NaN,0.113829,Use
1,BEV_S1_2024,2033,S1,D1,1070915,BEV_S1_2024,BEV,2024,177650,102000,D1,Electricity,0.000000,24871.0,33753.50,0.113829,Buy
1,BEV_S1_2024,2033,S1,D1,1070915,BEV_S1_2024,BEV,2024,177650,102000,D1,Electricity,0.000000,24871.0,33753.50,0.113829,Sell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,LNG_S3_2033,2033,S3,D4,262313,LNG_S3_2033,LNG,2033,203114,73000,D4,LNG,0.382737,10155.7,2031.14,0.146870,Sell
806,LNG_S3_2033,2033,S3,D4,262313,LNG_S3_2033,LNG,2033,203114,73000,D4,LNG,0.382737,10155.7,2031.14,0.146870,Use
807,LNG_S3_2033,2033,S3,D4,262313,LNG_S3_2033,LNG,2033,203114,73000,D4,BioLNG,0.058731,10155.7,2031.14,0.159213,Buy
807,LNG_S3_2033,2033,S3,D4,262313,LNG_S3_2033,LNG,2033,203114,73000,D4,BioLNG,0.058731,10155.7,2031.14,0.159213,Sell


In [502]:
def buy_costs(row):
    return row["Cost ($)"]
def use_costs():
    return None # it should return 0 if vehicle is present in previous years fleet
def resale_value(row):
    my_sql_operations = MySQLOperations() 
        
    query = f"""SELECT end_of_year, resale_value_percent FROM cost_profiles"""
    resale_value_data, columns = my_sql_operations.fetch_data(query)
    resale_value_df = pd.DataFrame(resale_value_data, columns=columns)

    eoy= row['Operating Year'] - row['Year'] + 1
    for index, _ in resale_value_df.iterrows():
        if _['end_of_year'] == eoy:
            resale_value = (_['resale_value_percent']/100) * row['Cost ($)']
    return resale_value

In [503]:
def calculate_cost(row):
    if row["Type"] == "Buy":
        return buy_costs(row)
    elif row["Type"] == "Use":
        return use_costs()
    elif row["Type"] == "Sell":
        return resale_value(row)

In [504]:
df_exploded["Cost"] = df_exploded.apply(calculate_cost, axis=1)

KeyError: 'Year'

In [ ]:
df_exploded.reset_index(drop=True, inplace=True)

In [ ]:
df_exploded

,Allocation,Year,Size,Distance_x,Demand (km),ID,Vehicle,Cost ($),Yearly range (km),Distance_y,Operating Year,Fuel,carbon_emissions,insurance_cost,maintenance_cost,fuel_costs,Type,Cost
0,BEV_S1_2023,2023,S1,D1,869181,BEV_S1_2023,BEV,187000,102000,D1,2023,Electricity,0.000000,9350.0,1870.00,17470.316221,Buy,187000.0
1,BEV_S1_2023,2023,S1,D1,869181,BEV_S1_2023,BEV,187000,102000,D1,2023,Electricity,0.000000,9350.0,1870.00,17470.316221,Sell,168300.0
2,BEV_S1_2023,2023,S1,D1,869181,BEV_S1_2023,BEV,187000,102000,D1,2023,Electricity,0.000000,9350.0,1870.00,17470.316221,Use,NaN
3,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,D4,2023,B20,69347.975963,4250.0,850.00,27771.219051,Buy,85000.0
4,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,D4,2023,B20,69347.975963,4250.0,850.00,27771.219051,Sell,76500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,LNG_S3_2023,2023,S3,D4,205426,LNG_S3_2023,LNG,151136,73000,D4,2023,LNG,29914.034489,7556.8,1511.36,12171.891758,Sell,136022.4
200,LNG_S3_2023,2023,S3,D4,205426,LNG_S3_2023,LNG,151136,73000,D4,2023,LNG,29914.034489,7556.8,1511.36,12171.891758,Use,NaN
201,LNG_S3_2023,2023,S3,D4,205426,LNG_S3_2023,LNG,151136,73000,D4,2023,BioLNG,4549.567924,7556.8,1511.36,13077.698068,Buy,151136.0
202,LNG_S3_2023,2023,S3,D4,205426,LNG_S3_2023,LNG,151136,73000,D4,2023,BioLNG,4549.567924,7556.8,1511.36,13077.698068,Sell,136022.4


In [ ]:
df_exploded.to_csv('data/something_new.csv', index=False)

In [ ]:
df = df_exploded.dropna(subset = ['Cost'])

In [ ]:
df.to_csv('data/something_new.csv', index=False)

In [ ]:
combinations = df.groupby(["Size", "Distance_x"])

In [ ]:
for (size, distance_x), group in combinations:
    

S1 D1
        Allocation  Year Size Distance_x  Demand (km)              ID Vehicle  \
0      BEV_S1_2023  2023   S1         D1       869181     BEV_S1_2023     BEV   
1      BEV_S1_2023  2023   S1         D1       869181     BEV_S1_2023     BEV   
3   Diesel_S1_2023  2023   S1         D1       869181  Diesel_S1_2023  Diesel   
4   Diesel_S1_2023  2023   S1         D1       869181  Diesel_S1_2023  Diesel   
6   Diesel_S1_2023  2023   S1         D1       869181  Diesel_S1_2023  Diesel   
7   Diesel_S1_2023  2023   S1         D1       869181  Diesel_S1_2023  Diesel   
9      LNG_S1_2023  2023   S1         D1       869181     LNG_S1_2023     LNG   
10     LNG_S1_2023  2023   S1         D1       869181     LNG_S1_2023     LNG   
12     LNG_S1_2023  2023   S1         D1       869181     LNG_S1_2023     LNG   
13     LNG_S1_2023  2023   S1         D1       869181     LNG_S1_2023     LNG   

    Cost ($)  Yearly range (km) Distance_y  Operating Year         Fuel  \
0     187000             10